In [1]:
'''
author: Felix Hol
date: 2019 June 11
Align bodypart coordinates, translation to put a specific bodypart at origin, rotation along body axis
'''

# import deeplabcut
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import tables
import glob
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pickle
from tqdm import tqdm
import seaborn as sns
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
# plt.style.use('seaborn-dark-palette')

behaviors:
1: rest
2: walk
3: probe
4: bite
5: imbibe
6: proboscis groom
7: hind leg groom
8: land
9: reorient
10: engorged

In [2]:
def getBodyAngle(dfRel, i):
    if dfRel[[('head','x')]].loc[i].values > 0 and dfRel[[('head','y')]].loc[i].values > 0:
        bodyAngleRaw = math.atan(dfRel[[('head','y')]].loc[i].values / dfRel[[('head','x')]].loc[i].values)
        bodyAngle = -1 * bodyAngleRaw
    elif dfRel[[('head','x')]].loc[i].values < 0 and dfRel[[('head','y')]].loc[i].values > 0:
        bodyAngleRaw = math.atan(dfRel[[('head','y')]].loc[i].values / (-1 * dfRel[[('head','x')]].loc[i].values))   
        bodyAngle = -1 * (math.pi - bodyAngleRaw)
    elif dfRel[[('head','x')]].loc[i].values < 0 and dfRel[[('head','y')]].loc[i].values < 0:
        bodyAngleRaw = math.atan((-1 * dfRel[[('head','y')]].loc[i].values) / (-1 * dfRel[[('head','x')]].loc[i].values))   
        bodyAngle = -1 * (math.pi + bodyAngleRaw)
    elif dfRel[[('head','x')]].loc[i].values > 0 and dfRel[[('head','y')]].loc[i].values < 0:
        bodyAngleRaw = math.atan((-1 * dfRel[[('head','y')]].loc[i].values) / (dfRel[[('head','x')]].loc[i].values))   
        bodyAngle = -1* (2 * math.pi - bodyAngleRaw)
    return bodyAngle


In [ ]:
dirList = glob.glob('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/*crops_*/')
alignTo = 'bottom'

dfRRSA = pd.DataFrame()

for k in tqdm(dirList):
    dataFile = glob.glob(k + '*labels.csv')
    df = pd.read_csv(dataFile[0], header=[0,1,2], index_col=0)
    df.columns = df.columns.droplevel()
    dfRel = df.copy()
    for i in dfRel.columns.levels[0]:
        if i != 'behavior':
            dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
            dfRel[i, 'y'] = dfRel[i, 'y'] - df[alignTo, 'y']
    dfRelRot = dfRel.copy()
    dfRelRot['','angle'] = ''
    for index, row in dfRelRot.iterrows():
        bodyAngle = getBodyAngle(dfRel, index)
        dfRelRot['', 'angle'][index] = bodyAngle
        for i in dfRelRot.columns.levels[0]:
            if i != 'behavior' and i != 'angle' and i != '':
                dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
                dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
    for i in dfRelRot.index:
        dfRelRot.loc[i, ('','frame')] = int(i[i.find('_f') + 2 : i.find('.png')])
    dfd = dfRelRot.copy()
    dfd.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in dfd.columns]
    dfd = dfd.rename(columns={"_frame": "frame"})
    dfd = dfd.rename(columns={"_angle": "angle"})
    dfd["angle"] = pd.to_numeric(dfd["angle"])
    dfd['smoothAngle'] = dfd['angle'].rolling(4).mean()
    diffAngle = np.diff(dfd['smoothAngle'])
    diffAngle = np.append(diffAngle, np.nan)
    dfd['diffAngle'] = diffAngle
        
    trackFile = glob.glob(k + '*tStats.pkl')
    trackStats = pickle.load(open(trackFile[0], 'rb'))
    
    mdf = pd.merge(dfd, trackStats, on='frame', how='inner')
    
    dfRRSA = dfRRSA.append(mdf, ignore_index=False)




In [ ]:
dfRRSA.head()

In [ ]:
dfRRSA.info()

In [ ]:
len(dfRRSA)

In [ ]:
dfRRSA = dfRRSA.dropna()

In [ ]:
len(dfRRSA)

In [ ]:
# dfC = dfRRSA.loc[:,~dfRRSA.columns.str.endswith('likelihood')]
#dfC = dfC.drop(columns=['x', 'y','age','species', 'angle', 'smoothAngle', 'behavior_behavior','frame', 'particle','distance', 'bottom_x', 'bottom_y'])
dfC = dfRRSA.drop(columns=['x', 'y','age','species', 'angle', 'smoothAngle', 'behavior_behavior','frame', 'particle','distance', 'bottom_x', 'bottom_y'])
dfCs = sklearn.preprocessing.StandardScaler(with_mean=True, with_std=True).fit_transform(dfC)

In [ ]:
len(dfRRSA[dfRRSA.behavior_behavior == 5])

In [ ]:
df1 = dfRRSA[dfRRSA.behavior_behavior == 10].sample(1000)
df2 = dfRRSA[~dfRRSA.index.isin(df1.index)]
df1

In [ ]:
len(df2)

In [ ]:
dfC.head()

In [ ]:
dfC.info()

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=15, n_iter=2000)
tsne_results = tsne.fit_transform(dfCs)

In [ ]:
tsnePlot = dfRRSA.copy()

tsnePlot['tsne-2d-one'] = tsne_results[:,0]
tsnePlot['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,13))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue=('behavior_behavior'),
    palette=sns.color_palette("hls", 8),
    data=dfRRSA,
    legend="full",
#     legend=['1','2'],
    alpha=0.5
)

In [ ]:
pca = PCA(n_components=2)
principalComponentsAll = pca.fit_transform(dfCs)
pcaDfAll = pd.DataFrame(data = principalComponentsAll, columns = ['pca1', 'pca2'])
plt.figure(figsize=(16,10))
plt.plot(pcaDfAll.pca1, pcaDfAll.pca2,'.', alpha=0.2)

In [ ]:
pcaPlot = dfRRSA.copy()

pcaPlot['pca1'] = pcaDfAll.pca1
pcaPlot['pca2'] = pcaDfAll.pca2
plt.figure(figsize=(16,13))
sns.scatterplot(
    x="pca1", y="pca2",
    hue=('behavior_behavior'),
    palette=sns.color_palette("hls", 8),
    data=pcaPlot,
    legend="full",
#     legend=['1','2'],
    alpha=0.2
)

In [ ]:
pcaPlot.head()

In [ ]:
pcaDfAll.head()

In [ ]:
X_test

In [ ]:
from sklearn.model_selection import train_test_split

X = dfCs.copy()
# X.columns = dfxy.columns.droplevel()
y = dfRRSA['behavior_behavior']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k = 8

knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train) 

In [ ]:
y_predict = knn_model.predict(X_test)
y_predict

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_predict))

In [ ]:
y_predict = knn_model.predict(dfC)
y_predict

In [ ]:
knn_model.predict(X)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_predict))

In [ ]:
dirList = glob.glob('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/190*p2/')
dirList

In [ ]:
dirList = glob.glob('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/No*p7/')
alignTo = 'bottom'

dfRRSA46 = pd.DataFrame()

for k in tqdm(dirList):
    dataFile = glob.glob(k + '*.csv')
    df = pd.read_csv(dataFile[0], header=[0,1,2], index_col=0)
    df.columns = df.columns.droplevel()
    dfRel = df.copy()
    for i in dfRel.columns.levels[0]:
        if i != 'behavior':
            dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
            dfRel[i, 'y'] = dfRel[i, 'y'] - df[alignTo, 'y']
    dfRelRot = dfRel.copy()
    dfRelRot['','angle'] = ''
    for index, row in dfRelRot.iterrows():
        bodyAngle = getBodyAngle(dfRel, index)
        dfRelRot['', 'angle'][index] = bodyAngle
        for i in dfRelRot.columns.levels[0]:
            if i != 'behavior' and i != 'angle' and i != '':
                dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
                dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
    for i in dfRelRot.index:
        dfRelRot.loc[i, ('','frame')] = int(i[i.find('_f') + 2 : i.find('.png')])
    dfd = dfRelRot.copy()
    dfd.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in dfd.columns]
    dfd = dfd.rename(columns={"_frame": "frame"})
    dfd = dfd.rename(columns={"_angle": "angle"})
    dfd["angle"] = pd.to_numeric(dfd["angle"])
    dfd['smoothAngle'] = dfd['angle'].rolling(4).mean()
    diffAngle = np.diff(dfd['smoothAngle'])
    diffAngle = np.append(diffAngle, np.nan)
    dfd['diffAngle'] = diffAngle
        
    trackFile = glob.glob(k + '*tStats.pkl')
    trackStats = pickle.load(open(trackFile[0], 'rb'))
    
    trackStats['bellyExp'] = np.nan
    trackStats['engorged'] = 0
    trackStats['fracEngorged'] = 0
    areaThreshold = 4750
    engorgementIncrease = 1.3

    if len(trackStats) > 100:
        if np.median(trackStats.bellyArea[0:25]) > areaThreshold:
                     trackStats['engorged'] = 2
        trackStats['bellyExp'] = trackStats.bellyArea.values / np.median(trackStats.bellyArea[0:99])
        trackStats['bellyExp'] = trackStats['bellyExp'].rolling(5).mean()
        if np.nanpercentile(trackStats['bellyExp'].values, 90) > engorgementIncrease:
            trackStats['fracEngorged'] = (trackStats['bellyExp'] - np.nanpercentile(trackStats['bellyExp'].values, 3)) / (np.nanpercentile(trackStats['bellyExp'].values, 90)- np.nanpercentile(trackStats['bellyExp'].values, 3))


    
    mdf = pd.merge(dfd, trackStats, on='frame', how='inner')
    
    dfRRSA46 = dfRRSA46.append(mdf, ignore_index=False)





In [ ]:
dfRRSA46 = dfRRSA46.dropna()
dfRRSA46.head()

In [ ]:
dfC46 = dfRRSA46.drop(columns=['x', 'y','age','species', 'angle', 'smoothAngle','frame', 'particle','distance', 'bottom_x', 'bottom_y'])
dfC46s = sklearn.preprocessing.StandardScaler(with_mean=True, with_std=True).fit_transform(dfC46)

In [ ]:
pPrediction = knn_model.predict(dfC46s)
pPrediction

In [ ]:
plt.plot(pPrediction)

In [ ]:
plt.plot(trackStats.bellyArea.values / np.median(trackStats.bellyArea[0:100]))

In [ ]:
np.median(trackStats.bellyArea[0:100])

In [ ]:
trackStats.head()

In [ ]:
len(tt)

In [ ]:
tt = trackStats.copy()
tt['bellyExp'] = np.nan
tt['engorged'] = 0
tt['fracEngorged'] = 0
areaThreshold = 4750
engorgementIncrease = 1.3

if len(tt) > 100:
    if np.median(trackStats.bellyArea[0:25]) > areaThreshold:
                 tt['engorged'] = 2
    tt['bellyExp'] = trackStats.bellyArea.values / np.median(trackStats.bellyArea[0:99])
    tt['bellyExp'] = tt['bellyExp'].rolling(5).mean()
    if np.nanpercentile(tt['bellyExp'].values, 90) > engorgementIncrease:
        tt['fracEngorged'] = (tt['bellyExp'] - np.nanpercentile(tt['bellyExp'].values, 3)) / (np.nanpercentile(tt['bellyExp'].values, 90)- np.nanpercentile(tt['bellyExp'].values, 3))
        
    



In [ ]:
plt.plot(trackStats.bellyExp.values)

In [ ]:
plt.plot(trackStats.fracEngorged.values)

In [ ]:
from sklearn.model_selection import train_test_split

X = dfCs.copy()
# X.columns = dfxy.columns.droplevel()
y = dfRRSA['behavior_behavior']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k = 8

knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train) 

In [ ]:
y_predict = knn_model.predict(dfCs)
y_predict

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_predict))

In [ ]:
len(dfRRSA46)

In [ ]:
dfRRSA46 = dfRRSA46.dropna()

len(dfRRSA46)

In [ ]:
# dfC = dfRRSA.loc[:,~dfRRSA.columns.str.endswith('likelihood')]
#dfC = dfC.drop(columns=['x', 'y','age','species', 'angle', 'smoothAngle', 'behavior_behavior','frame', 'particle','distance', 'bottom_x', 'bottom_y'])
dfC46 = dfRRSA46.drop(columns=['x', 'y','age','species', 'angle', 'smoothAngle','frame', 'particle','distance', 'bottom_x', 'bottom_y'])
dfC46s = sklearn.preprocessing.StandardScaler(with_mean=True, with_std=True).fit_transform(dfC46)

In [ ]:
dfC46.info()

In [ ]:
dfC.info()

In [ ]:
y_predict = knn_model.predict(dfC46s)
y_predict

In [ ]:
plt.plot(y_predict)

In [ ]:
plt.plot(dfC46.bellyArea.values)

In [ ]:
dataFile = glob.glob(dirList[0] + '*labels.csv')
df = pd.read_csv(dataFile[0], header=[0,1,2], index_col=0)
df.columns = df.columns.droplevel()
for i in df.index:
    df.loc[i, ('','frame')] = int(i[i.find('_f') + 2 : i.find('.png')])

In [ ]:
fileList = glob.glob('/Users/felix/moreTest01-felix-2019-05-07/analyzeVideos/*DeepCut_resnet50_moreTest01May6shuffle1_65700.h5')
alignTo = 'bottom'

dfRelRotAll = pd.DataFrame()

for i in fileList:
    dataFile = i
    df = pd.read_hdf(dataFile)
    df.columns = df.columns.droplevel()
    dfRel = df.copy()
    for i in dfRel.columns.levels[0]:
        dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
        dfRel[i, 'y'] = dfRel[i, 'y'] - df[alignTo, 'y']
    dfRelRot = dfRel.copy()
    for index, row in dfRelRot.iterrows():
        bodyAngle = getBodyAngle(dfRel, index)
        for i in dfRelRot.columns.levels[0]:
            dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
            dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
    dfRelRotAll = dfRelRotAll.append(dfRelRot, ignore_index=False)


In [ ]:
dirList = glob.glob('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/*crops_*/')

In [ ]:
dirList

In [ ]:
dfRelRot = dfRel.copy()
dfRelRot['','angle'] = ''

for index, row in dfRelRot.iterrows():
#     print(index)
    bodyAngle = getBodyAngle(dfRel, index)
#     dfRelRot.loc[index,('bodyparts','angle')] = bodyAngle
    dfRelRot['', 'angle'][index] = bodyAngle
#     print(bodyAngle)
    for i in dfRelRot.columns.levels[0][0:10]:
#         print(i)
#         print(dfRel[i, 'x'][index])
        dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
        dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
#         print(bodyAngle)
#         print(dfRel[i, 'x'][index])

In [ ]:
fileList = glob.glob('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/*crops_*/*.h5')
alignTo = 'bottom'

dfRelRotAll = pd.DataFrame()

for i in fileList:
    dataFile = i
    df = pd.read_hdf(dataFile)
    df.columns = df.columns.droplevel()
    dfRel = df.copy()
    for i in dfRel.columns.levels[0]:
        dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
        dfRel[i, 'y'] = dfRel[i, 'y'] - df[alignTo, 'y']
    dfRelRot = dfRel.copy()
    for index, row in dfRelRot.iterrows():
        bodyAngle = getBodyAngle(dfRel, index)
        for i in dfRelRot.columns.levels[0]:
            dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
            dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
    dfRelRotAll = dfRelRotAll.append(dfRelRot, ignore_index=False)



In [ ]:
fileList = glob.glob('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/*crops_p*/*labels.csv')
alignTo = 'bottom'

dfRelRotAll = pd.DataFrame()

for i in fileList:
    dataFile = i
    df = pd.read_csv (dataFile, header=[0,1,2], index_col=0)
    df.columns = df.columns.droplevel()
    dfRel = df.copy()
    for i in dfRel.columns.levels[0]:
        if i != 'behavior':
            dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
            dfRel[i, 'y'] = dfRel[i, 'y'] - df[alignTo, 'y']
    dfRelRot = dfRel.copy()
    for index, row in dfRelRot.iterrows():
        bodyAngle = getBodyAngle(dfRel, index)
        for i in dfRelRot.columns.levels[0]:
            if i != 'behavior':
                dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
                dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
    dfRelRotAll = dfRelRotAll.append(dfRelRot, ignore_index=False)




In [ ]:
dfRelRot.head()

In [ ]:
csvFile = '/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/forKNN/NoATP_M2_crops_p27/crops_p27DeepCut_resnet50_moreTest01Jul23shuffle1_244500.csv'
df2 = pd.read_csv (csvFile, header=[0,1,2], index_col=0)
df2.columns = df2.columns.droplevel()

In [ ]:
df2.head()

In [ ]:
dataFile

In [ ]:
len(dfRelRot)

In [ ]:
dfRelRot.head()

In [ ]:
len(dfRelRot)

In [ ]:
dfRelRotAll.head()

In [ ]:
len(dfRelRotAll)

In [ ]:
dfRRA = dfRelRotAll.copy()
for i in dfRel.columns.levels[0]:
    if i != 'behavior':
#     dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
        dfRRA = dfRRA.drop(dfRRA[dfRRA[(i,'likelihood')] < 0.9].index)
# dfRRA = dfRRA.drop(dfRRA[columns=['likelihood'], level=1 ].index)

In [ ]:
i

In [ ]:
len(dfRRA)

In [ ]:
dfRRA[('rightForeleg','likelihood')]

In [ ]:
ax = dfRelRotAll.plot(legend=False,figsize=(15,15))
ax.grid(False)
# plt.xlim([2500, 3000])

In [ ]:
ax = dfRRA.plot(legend=False,figsize=(15,15))
ax.grid(False)

In [ ]:
tp13 = pickle.load(open('/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/190702_SalineNoATP_M3/190702_SalineNoATP_M3_p13_tStats.pkl', 'rb'))

In [ ]:
tp13.head()

In [ ]:
dataFile = '/Users/felix/Documents/mosquitoes/mosquitoData/bloodTasteData/trackAndBodypart/190702_SalineNoATP_M3/crops_p13/crops_p13DeepCut_resnet50_moreTest01Jul23shuffle1_244500.h5'
df = pd.read_hdf(dataFile)
df.columns = df.columns.droplevel()
# df['','frame'] = ''

In [ ]:
for i in df.index:
    df.loc[i, ('','frame')] = int(i[i.find('_f') + 2 : i.find('.png')])
    

In [ ]:
alignTo = 'bottom'
dfRel = df.copy()

for i in dfRel.columns.levels[0]:
    dfRel[i, 'x'] = dfRel[i, 'x'] - df[alignTo, 'x']
    dfRel[i, 'y'] = dfRel[i, 'y'] - df[alignTo, 'y']

In [ ]:
dfRel.head()

In [ ]:
dfRelRot = dfRel.copy()
dfRelRot['','angle'] = ''

for index, row in dfRelRot.iterrows():
#     print(index)
    bodyAngle = getBodyAngle(dfRel, index)
#     dfRelRot.loc[index,('bodyparts','angle')] = bodyAngle
    dfRelRot['', 'angle'][index] = bodyAngle
#     print(bodyAngle)
    for i in dfRelRot.columns.levels[0][0:10]:
#         print(i)
#         print(dfRel[i, 'x'][index])
        dfRelRot[i, 'x'][index] = (dfRel[i, 'x'][index] * math.cos(bodyAngle)) - (dfRel[i, 'y'][index] * math.sin(bodyAngle))
        dfRelRot[i, 'y'][index] = (dfRel[i, 'x'][index] * math.sin(bodyAngle)) + (dfRel[i, 'y'][index] * math.cos(bodyAngle))
#         print(bodyAngle)
#         print(dfRel[i, 'x'][index])

In [ ]:
len(dfRelRot.columns.levels[0])

In [ ]:
dfRelRot.columns.levels[0][0:11]

In [ ]:
dfRelRot.head()

In [ ]:
bodyAngle

In [ ]:
row

In [ ]:
dfRelRot['','frame'] = ''

for i in dfRelRot.index:
    dfRelRot.loc[i, ('','frame')] = int(i[i.find('_f') + 2 : i.find('.png')])

In [ ]:
dfd = dfRelRot.copy()
dfd.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in dfd.columns]
dfd = dfd.rename(columns={"_frame": "frame"})
dfd = dfd.rename(columns={"_angle": "angle"})
dfd["angle"] = pd.to_numeric(dfd["angle"])

In [ ]:
# dfd = df.copy()
# dfd.columns = dfd.columns.droplevel()
mdf = pd.merge(dfd, tp13, on='frame', how='inner')

In [ ]:
mdf.tail()

In [ ]:
mdf.info()

In [ ]:
mdf.angle

In [ ]:
dfD = mdf.drop(columns=['frame', 'species', 'age'])
dfD = dfD.dropna()
dfD.head()

In [ ]:
len(dfD)

In [ ]:
dfNoBeh = dfRRAnan.drop(columns=['behavior'], level=1)

In [ ]:
dfNoBeh = dfRelRotAll.drop(columns=['behavior'], level=1)
dfNoBeh = dfNoBeh.dropna()

In [ ]:
dfNoBeh['tsne-2d-one'] = tsne_resultsNoBeh[:,0]
dfNoBeh['tsne-2d-two'] = tsne_resultsNoBeh[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
#     hue=("behavior",'behavior'),
#     palette=sns.color_palette("hls", 5),
    data=dfNoBeh,
    legend="full",
#     legend=['1','2'],
    alpha=0.99
)

In [ ]:
dfStrip = dfRRAnan.copy()
dfStrip.columns = dfRRAnan.columns.droplevel()

In [ ]:
dfStrip.head()

In [ ]:
len(dfStrip[dfStrip['behavior'] == 4])

In [ ]:
dfRRAnan[dfRRAnan[('behavior', 'behavior')] == 4]

In [ ]:
tsne3 = TSNE(n_components=3, verbose=1, perplexity=30, n_iter=3000)
dfCs = sklearn.preprocessing.StandardScaler(with_mean=True, with_std=True).fit_transform(dfC)
tsne_results3 = tsne3.fit_transform(dfCs)

In [ ]:
dfRRSA['tsne-3d-one'] = tsne_results3[:,0]
dfRRSA['tsne-3d-two'] = tsne_results3[:,1]
dfRRSA['tsne-3d-three'] = tsne_results3[:,2]
plt.figure(figsize=(16,13))
sns.scatterplot(
    x="tsne-3d-one", y="tsne-3d-two", s="tsne-3d-three",
    hue=('behavior_behavior'),
    palette=sns.color_palette("hls", 6),
    data=dfRRSA,
#     legend="full",
# #     legend=['1','2'],
#     alpha=0.99
)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(tsne_results3[:,0], tsne_results3[:,1], tsne_results3[:,2],  s=50, alpha=0.6, edgecolors='w')

In [ ]:
dfRRAnan = dfRRA.dropna()
# tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=2000)
# tsne_results = tsne.fit_transform(dfRRAnan, y=('behavior', 'behavior'))

In [ ]:
dfRRAnan.head()

In [ ]:
dfRRAnan['tsne-2d-one'] = tsne_results[:,0]
dfRRAnan['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue=("behavior",'behavior'),
    palette=sns.color_palette("hls", 5),
    data=dfRRAnan,
    legend="full",
#     legend=['1','2'],
    alpha=0.99
)

# plt.legend(['1','2','3','4','5','6','7','8','9'])

# plt.legend(['1','2','3','4'])

behaviors:
1: rest
2: walk
3: probe
4: bite
5: imbibe
6: proboscis groom
7: hind leg groom
8: land
9: reorient

In [ ]:
pca = PCA(n_components=2)
principalComponentsAll = pca.fit_transform(dfCs)
pcaDfAll = pd.DataFrame(data = principalComponentsAll, columns = ['pca1', 'pca2'])
plt.figure(figsize=(16,10))
plt.plot(pcaDfAll.pca1, pcaDfAll.pca2,'.', alpha=0.2)

In [ ]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=2)
pca_model.fit(X_train)
X_train = pca_model.transform(X_train)
X_test = pca_model.transform(X_test)

# 2-Dimensions
X_train[:5]

In [ ]:
# ax = plt.figure()
ax = dfRelRot.plot(legend=False,figsize=(15,15))
ax.grid(False)
# plt.xlim([190, 380])

In [ ]:
dfRelRot.head()

In [ ]:
plt.plot(dfRelRot[[('head','x')]].loc[0], dfRelRot[[('head','y')]].loc[0],'.r')

In [ ]:
plt.plot(df[[('head','x')]].loc[0], df[[('head','y')]].loc[0],'.r')
plt.plot(df[[('abdomen','x')]].loc[0], df[[('abdomen','y')]].loc[0],'.g')
plt.plot(df[[('bottom','x')]].loc[0], df[[('bottom','y')]].loc[0],'.k')
plt.plot(df[[('rightForeleg','x')]].loc[0], df[[('rightForeleg','y')]].loc[0],'.b')
plt.plot(df[[('leftForeleg','x')]].loc[0], df[[('leftForeleg','y')]].loc[0],'.m')

In [ ]:
plt.plot(dfRel[[('head','x')]].loc[0], dfRel[[('head','y')]].loc[0],'.r')
plt.plot(dfRel[[('abdomen','x')]].loc[0], dfRel[[('abdomen','y')]].loc[0],'.g')
plt.plot(dfRel[[('bottom','x')]].loc[0], dfRel[[('bottom','y')]].loc[0],'.k')
plt.plot(dfRel[[('rightForeleg','x')]].loc[0], dfRel[[('rightForeleg','y')]].loc[0],'.b')
plt.plot(dfRel[[('leftForeleg','x')]].loc[0], dfRel[[('leftForeleg','y')]].loc[0],'.m')

In [ ]:
plt.plot(dfRelRot[[('head','x')]].loc[0], dfRelRot[[('head','y')]].loc[0],'.r')
plt.plot(dfRelRot[[('abdomen','x')]].loc[0], dfRelRot[[('abdomen','y')]].loc[0],'.g')
plt.plot(dfRelRot[[('bottom','x')]].loc[0], dfRelRot[[('bottom','y')]].loc[0],'.k')
plt.plot(dfRelRot[[('rightForeleg','x')]].loc[0], dfRelRot[[('rightForeleg','y')]].loc[0],'.b')
plt.plot(dfRelRot[[('leftForeleg','x')]].loc[0], dfRelRot[[('leftForeleg','y')]].loc[0],'.m')

In [ ]:
plt.plot(dfRelRot[[('head','x')]].loc[:], dfRelRot[[('head','y')]].loc[:],'.w')
plt.plot(dfRelRot[[('abdomen','x')]].loc[:], dfRelRot[[('abdomen','y')]].loc[:],'.w')
plt.plot(dfRelRot[[('bottom','x')]].loc[:], dfRelRot[[('bottom','y')]].loc[:],'.w')
plt.plot(dfRelRot[[('rightForeleg','x')]].loc[:], dfRelRot[[('rightForeleg','y')]].loc[:],'.b')
plt.plot(dfRelRot[[('leftForeleg','x')]].loc[:], dfRelRot[[('leftForeleg','y')]].loc[:],'.m')
plt.plot(dfRelRot[[('rightHindLeg','x')]].loc[:], dfRelRot[[('rightHindLeg','y')]].loc[:],'.c')
plt.plot(dfRelRot[[('leftHindLeg','x')]].loc[:], dfRelRot[[('leftHindLeg','y')]].loc[:],'.y')
plt.plot(dfRelRot[[('rightMiddleLeg','x')]].loc[:], dfRelRot[[('rightMiddleLeg','y')]].loc[:],'.k')
plt.plot(dfRelRot[[('leftMiddleLeg','x')]].loc[:], dfRelRot[[('leftMiddleLeg','y')]].loc[:],'.k')
plt.plot(dfRelRot[[('proboscis','x')]].loc[:], dfRelRot[[('proboscis','y')]].loc[:],'.r')

In [ ]:
dfRelRot.head()

In [ ]:
plt.plot(dfRelRot[[('head','x')]].loc[:], dfRelRot[[('head','y')]].loc[:],'w')
# plt.plot(dfRelRot[[('abdomen','x')]].loc[:], dfRelRot[[('abdomen','y')]].loc[:],'w')
# plt.plot(dfRelRot[[('bottom','x')]].loc[:], dfRelRot[[('bottom','y')]].loc[:],'w')
plt.plot(dfRelRot[[('rightForeleg','x')]].loc[:], dfRelRot[[('rightForeleg','y')]].loc[:],'b')
plt.plot(dfRelRot[[('leftForeleg','x')]].loc[:], dfRelRot[[('leftForeleg','y')]].loc[:],'m')
# plt.plot(dfRelRot[[('rightHindLeg','x')]].loc[:], dfRelRot[[('rightHindLeg','y')]].loc[:],'c')
# plt.plot(dfRelRot[[('leftHindLeg','x')]].loc[:], dfRelRot[[('leftHindLeg','y')]].loc[:],'y')
# plt.plot(dfRelRot[[('rightMiddleLeg','x')]].loc[:], dfRelRot[[('rightMiddleLeg','y')]].loc[:],'k')
# plt.plot(dfRelRot[[('leftMiddleLeg','x')]].loc[:], dfRelRot[[('leftMiddleLeg','y')]].loc[:],'g')
plt.plot(dfRelRot[[('proboscis','x')]].loc[:], dfRelRot[[('proboscis','y')]].loc[:],'r')

In [ ]:
plt.plot(dfRelRot[[('proboscis','x')]].loc[:], dfRelRot[[('proboscis','y')]].loc[:],'r')

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(df[[('rightMiddleLeg','x')]], 'c')
plt.plot(df[[('leftMiddleLeg','x')]], 'm')
plt.plot(df[[('rightMiddleLeg','y')]], '--c')#, alpha=0.25)
plt.plot(df[[('leftMiddleLeg','y')]], '--m')#, alpha=0.25)
plt.plot(df[[('proboscis','x')]], 'k')
plt.plot(df[[('proboscis','y')]], '--k')
# plt.plot(df[[('proboscis','y')]], '--k')#, alpha=0.25)
# plt.plot(df[[('head','x')]], 'w')
# plt.plot(df[[('head','y')]], '--w')
# plt.plot(df[[('bottom','x')]], 'k', alpha=0.25)
# plt.plot(df[[('bottom','y')]], 'k', alpha=0.25)
# plt.plot(df[[('bottom','x')]], '--k')
# plt.plot(df[[('bottom','y')]], 'k')
plt.plot(df[[('abdomen','x')]], 'y')
plt.plot(df[[('abdomen','y')]], '--y')
# plt.plot(df[[('rightMiddleLeg','y')]], 'w')
# plt.plot(df[[('leftForeleg','x')]], 'm')

# plt.plot(df[[('rightMiddleLeg','y')]], 'w')



plt.xlabel('time (frames)')
plt.ylabel('body part position (A.U.)')
plt.legend(['right middle leg, x', 'left middle leg, x', 'right middle leg, y', 'left middle leg, y', 'proboscis, x', 'proboscis, y', 'abdomen, x', 'abdomen, y'])
# plt.xlim((-2, 45))
# plt.savefig('181206_01_1783-1816_exploring_01.pdf')

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(dfRel[[('rightMiddleLeg','x')]], 'c')
plt.plot(dfRel[[('leftMiddleLeg','x')]], 'm')
plt.plot(dfRel[[('rightMiddleLeg','y')]], '--c')#, alpha=0.25)
plt.plot(dfRel[[('leftMiddleLeg','y')]], '--m')#, alpha=0.25)
plt.plot(dfRel[[('proboscis','x')]], 'k')
plt.plot(dfRel[[('proboscis','y')]], '--k')
# plt.plot(df[[('proboscis','y')]], '--k')#, alpha=0.25)
# plt.plot(df[[('head','x')]], 'w')
# plt.plot(df[[('head','y')]], '--w')
# plt.plot(df[[('bottom','x')]], 'k', alpha=0.25)
# plt.plot(df[[('bottom','y')]], 'k', alpha=0.25)
# plt.plot(df[[('bottom','x')]], '--k')
# plt.plot(df[[('bottom','y')]], 'k')
plt.plot(dfRel[[('abdomen','x')]], 'y')
plt.plot(dfRel[[('abdomen','y')]], '--y')
# plt.plot(df[[('rightMiddleLeg','y')]], 'w')
# plt.plot(df[[('leftForeleg','x')]], 'm')

# plt.plot(df[[('rightMiddleLeg','y')]], 'w')



plt.xlabel('time (frames)')
plt.ylabel('body part position (A.U.)')
plt.legend(['right middle leg, x', 'left middle leg, x', 'right middle leg, y', 'left middle leg, y', 'proboscis, x', 'proboscis, y', 'abdomen, x', 'abdomen, y'])
# plt.xlim((-2, 45))
# plt.savefig('181206_01_1783-1816_exploring_01.pdf')

In [ ]:
plt.figure(figsize=(15,11))
plt.plot(dfRelRot[[('rightMiddleLeg','x')]], 'y')
# plt.plot(dfRelRot[[('leftMiddleLeg','x')]], 'm')
# plt.plot(dfRelRot[[('rightHindLeg','x')]], 'm')
plt.plot(dfRelRot[[('rightHindLeg','y')]], 'm')
plt.plot(dfRelRot[[('rightForeleg','x')]], 'c')#, alpha=0.25)
# plt.plot(dfRelRot[[('rightForeleg','y')]], '--m')#, alpha=0.25)
# plt.plot(dfRelRot[[('proboscis','x')]], 'k')
plt.plot(dfRelRot[[('proboscis','y')]], 'k')
# plt.plot(df[[('proboscis','y')]], '--k')#, alpha=0.25)
# plt.plot(df[[('head','x')]], 'w')
# plt.plot(df[[('head','y')]], '--w')
# plt.plot(df[[('bottom','x')]], 'k', alpha=0.25)
# plt.plot(df[[('bottom','y')]], 'k', alpha=0.25)
# plt.plot(df[[('bottom','x')]], '--k')
# plt.plot(df[[('bottom','y')]], 'k')
# plt.plot(dfRelRot[[('abdomen','x')]], 'y')
# plt.plot(dfRelRot[[('abdomen','y')]], '--y')
# plt.plot(df[[('rightMiddleLeg','y')]], 'w')
# plt.plot(df[[('leftForeleg','x')]], 'm')

# plt.plot(df[[('rightMiddleLeg','y')]], 'w')



plt.xlabel('time (frames)')
plt.ylabel('body part position (A.U.)')
plt.legend(['right middle leg, x', 'right hind leg, y', 'right foreleg, x', 'proboscis, y'])#, 'proboscis, x', 'proboscis, y', 'abdomen, x', 'abdomen, y'])
# plt.xlim((-2, 45))
# plt.savefig('181206_01_1783-1816_exploring_01.pdf')